In [7]:
import pandas as pd
import numpy as np

In [8]:
# Load the data
data0  =  pd.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet\partition_id=8/part-0.parquet')
data1  =  pd.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet\partition_id=9/part-0.parquet')

In [9]:
# remove the nan values
def replace_nan_with_mean_or_zero(column):
    mean_value = column.mean()
    if pd.notnull(mean_value):
        return column.fillna(mean_value)
    else:
        return column.fillna(0)

In [10]:
train_data = pd.concat([data0, data1], axis=0)  # 2 partitions
train_data.apply(replace_nan_with_mean_or_zero)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,1360,0,0,1.865621,2.891473,0.353332,1.668053,2.853531,3.412320,-1.245219,...,-0.352222,0.502829,0.115287,-0.313245,0.651638,-1.308276,-1.620408,0.374767,-1.049378,-2.496996
1,1360,0,1,1.823830,3.157964,0.191611,2.494816,2.845078,3.090568,-0.742658,...,-0.212594,0.351176,-0.131981,0.127530,0.053275,-0.241975,-0.027899,-0.184599,-0.216430,-0.182844
2,1360,0,2,1.411181,2.716050,0.036010,1.966540,2.863158,2.959575,-1.340717,...,3.219775,-0.883570,-0.445202,0.102318,-0.391579,-0.871908,0.363652,0.236567,-1.445096,0.506134
3,1360,0,3,1.544636,2.704180,0.410197,2.214327,3.090716,3.155712,-0.923788,...,0.281109,-0.524510,-0.459967,-0.971227,-0.524694,0.761837,-1.442856,-0.447208,0.857494,-1.468352
4,1360,0,4,1.061685,2.375254,0.886124,2.317989,3.283024,2.975022,-1.422651,...,0.217880,-0.623650,-0.370462,0.063419,-0.597050,-0.952827,-3.025858,-0.506223,-0.317776,-4.478455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274571,1698,967,34,3.242493,2.525160,-0.721981,2.544025,2.477615,0.417557,0.785812,...,0.016936,0.243475,0.166927,0.384940,-0.174297,-0.066046,-0.038767,-0.132337,-0.022426,-0.252461
6274572,1698,967,35,1.079139,1.857906,-0.790646,2.745439,2.339877,0.845065,0.651370,...,0.050860,0.850152,0.909382,1.015314,0.235962,0.122539,0.099559,-0.249584,-0.123571,-0.460630
6274573,1698,967,36,1.033172,2.515527,-0.672298,2.289250,2.521592,0.255077,0.919892,...,0.152333,0.395684,-0.292574,-3.215846,-0.535129,-0.178484,-1.808150,-0.065355,-0.000367,-0.125170
6274574,1698,967,37,1.243116,2.663298,-0.889112,2.313155,3.101428,0.324454,0.618944,...,-0.029483,1.925987,0.479394,3.621867,-0.107114,-0.063599,1.204755,-0.148711,-0.026583,-0.256395


In [11]:
# define the columns
lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]

In [12]:
#new lag data
lags = train_data[lag_cols_original]
lags = lags.rename(columns= { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)})

lags.date_id = lags.date_id + 1
lags = lags.groupby(['date_id', 'symbol_id']).last().reset_index()  # pick up last record of previous date
train_data = train_data.merge(lags, on = ['date_id', 'symbol_id'], how = 'left')
train_data =  train_data.fillna(0)

In [17]:
num_val = int(len(train_data.date_id.unique())*0.05)

In [18]:
# Split the data into training and validation sets
max_date_id = train_data['date_id'].max()

train_set = train_data[train_data['date_id'] < max_date_id - num_val]
val_set = train_data[train_data['date_id'] >= max_date_id - num_val]

In [19]:
train_set.to_parquet('data/train_data.parquet')
val_set.to_parquet('data/validate_data.parquet')